In [1]:
import numpy as np
import csv
import pandas as pd
import time

### 데이터를 불러와서 학습할 수 있는 형태로 변환해주는 단계

In [2]:
# 학습에 활용할 최초의 데이터를 불러오는 작업
with open('abalone.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None)
    rows = []
    for row in csvreader:
        rows.append(row)

# 인풋과 아웃풋의 차원을 설정해 주고 data를 텐서화 시켜주는 작업
input_cnt , output_cnt = 10,1
data = np.zeros([len(rows),input_cnt+output_cnt])

# 불러온 데이터를 0으로 채워진 텐서구조에 넣어주기
for n, row in enumerate(rows):
    if row[0] == "I" : data[n,0] = 1
    if row[0] == "M" : data[n,1] = 1
    if row[0] == "F" : data[n,2] = 1
    data[n,3:] = row[1:]

### 데이터 학습 및 평가

In [17]:
# 초기 weight,bias를 설정해줌 : 정규분포를 갖는 난수값으로 초기 설정을 해줌)
# 정규분포 난수값의 평균과 표준편차를 설정하여 가중치의 파라미터를 초기화 할때 사용한다
RND_MEAN = 0
RND_STD = 0.0030

# 학습률을 설정(하이퍼파라미터)
LEARNING_LATE = 0.001

weight = np.random.normal(RND_MEAN,RND_STD,[input_cnt , output_cnt])
bias = np.zeros([output_cnt])

# 에포크 횟수(전체 학습횟수)와 미니배치 크기(한번에 학습하는 크기)를 설정하고 리포트 보고 주기를 설정
epoch_count = 10
mb_size = 10
report = 1

# 데이터를 섞고 랜덤으로 추출하여 학습하기 위한 데이터를 준비
shuffle_map = np.arange(data.shape[0])                   # 데이터하나하나에 순번을 붙여주는 작업
np.random.shuffle(shuffle_map)                           # 순번을 섞는 작업
step_count = int(data.shape[0] * 0.8) // mb_size         # 데이터를 한번에 학습할 크기로 전체크기를 나눠 전체를 학습하기 위해 몇회를 해야되는지의 값
test_begin_idx = step_count * mb_size                    # 테스트 데이터가 시작될 시작점의 번호(해당 인덱스를 기준으로 학습데이터와 테스트 데이터를 나누면됨)

# 데이터를 학습및 테스트
for epoch in range(epoch_count):
    losses, accs = [], []

    for n in range(step_count):
        if n == 0:
            np.random.shuffle(shuffle_map[:test_begin_idx])                   # 에포크가 한번 끝나고 다시 학습을 할때마다 데이터를 섞어준다, 데이터는 8:2로 나눠져있으며 인덱스값으로 구분
        train_x = data[shuffle_map[mb_size*n:mb_size*(n+1)]][:, :-output_cnt] # 데이터에서 가장 마지막값을 제외한 나머지 값이 학습 데이터이다 => x
        train_y = data[shuffle_map[mb_size*n:mb_size*(n+1)]][:, -output_cnt:] # 데이터에서 가장 마지막값은 정답인 정답데이터 이다 => y
        
        
        # forward path 연산
        output_train_x = np.matmul(train_x,weight) + bias
        
        diff = output_train_x - train_y
        loss = np.mean(np.square(diff))

        # step마다 loss와acc를 계산
        accuracy = 1 - np.mean(np.abs((output_train_x - train_y)/train_y))
        losses.append(loss)
        accs.append(accuracy)

        # backporpagation 연산(weight와 bias 업데이트)
        G_loss = 1.0
        
        g_loss_suqare = np.ones(diff.shape) / np.prod(diff.shape)
        g_square_diff = 2*diff
        g_diff_outout = 1

        G_square = g_loss_suqare * G_loss
        G_diff = g_square_diff * G_square
        G_output = g_diff_outout * G_diff

        g_outout_w = train_x.transpose()

        G_w = np.matmul(g_outout_w,G_output)
        G_b = np.sum(G_output, axis=0)

        weight -= LEARNING_LATE * G_w
        bias -= LEARNING_LATE * G_b

    if report > 0 and (epoch+1) % report == 0:
        test_data = data[shuffle_map[test_begin_idx:]]
        test_x = test_data[:, :-output_cnt]
        test_y = test_data[:, -output_cnt:]
        output_test_x = np.matmul(test_x,weight) + bias
        acc= 1 - np.mean(np.abs((output_test_x - test_y)/test_y))
        print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
              format(epoch+1,np.mean(losses),np.mean(accs),acc))

output_test_x = np.matmul(test_x,weight) + bias
final_acc= 1 - np.mean(np.abs((output_test_x - test_y)/test_y))
print('\nFinal Test : final accuracy = {:5.3f}'.format(final_acc))      

Epoch 1: loss=33.602, accuracy=0.557/0.807
Epoch 2: loss=8.136, accuracy=0.819/0.813
Epoch 3: loss=7.475, accuracy=0.813/0.809
Epoch 4: loss=7.359, accuracy=0.811/0.808
Epoch 5: loss=7.271, accuracy=0.810/0.809
Epoch 6: loss=7.201, accuracy=0.811/0.808
Epoch 7: loss=7.140, accuracy=0.810/0.809
Epoch 8: loss=7.086, accuracy=0.811/0.809
Epoch 9: loss=7.036, accuracy=0.810/0.810
Epoch 10: loss=6.997, accuracy=0.811/0.810

Final Test : final accuracy = 0.810
